In [0]:
import tensorflow as tf
import numpy as np

In [0]:
graph = tf.Graph()
session = tf.InteractiveSession(graph=graph)
# WE WANT TO USE TF TO COMPUTE h = sigmoid(w*x + b)

In [0]:
# Placeholders are not initialized with some value
# Rather we will provide the value on the fly at the time of graph execution
x = tf.placeholder(shape=[1, 10], dtype=tf.float32, name='x')

In [0]:
# Variables are mutable; their values can change over time
W = tf.Variable(tf.random_uniform(shape=[10, 5], minval=-0.1, maxval=0.1, dtype=tf.float32), name='w')
b = tf.Variable(tf.zeros(shape=[5], dtype=tf.float32), name='b')
# Arguments are variable initializers and are the tensors that will be assigned to the W and b variables initially
# Variables need initial values as placeholders and need to have some value assigned to them at all times
# tf.random_uniform means we uniformly sample values between minval=-0.1 and maxval=0.1 to assign values to the tensors
# tf.zeros initializes the tensor with zeros
# shape defines the size of each dimension of a tensor

In [0]:
# h is an immutable tensor produced by performing some operations on x, W, and b
h = tf.nn.sigmoid(tf.matmul(x, W) + b)

In [0]:
# Initialization operation that initializes the variables (W and b) in the graph
tf.global_variables_initializer().run()

In [0]:
# Execute the graph to obtain the final output, h
h_eval = session.run(h, feed_dict={x: np.random.rand(1, 10)})

In [0]:
# Close the session
session.close()

In [0]:
h_eval

array([[0.5026527 , 0.4800755 , 0.5489332 , 0.46823615, 0.5351664 ]],
      dtype=float32)

In any client program written with TensorFlow, there will be two main types of objects:
1. Operations
2. Tensors

In example above, ```tf.nn.sigmoid``` is an operation and ```h``` is a tensor.
Then we have a ```graph``` object, which is the computational graph that stores the dataflow of our program. 
When we add the subsequent lines defining ```x, W, b```and ```h``` in the code, TensorFlow automatically adds these tensors to the graph.
Next, session executes graph by dividing graph into subgraphs and subsequently to even finer pieces which will then be assigned to workers that will perform the assigned task. Done with the ```session.run()``` function.

Most common elements that comprise a TensorFlow client:
  1. **Inputs**: Data used to train and test our algorithms
  2. **Variables**: Mutable tensors, mostly defining the parameters of our algorithms
  3. **Outputs**: Immutable tensors storing both terminal and intermediate outputs
  4. **Operations**: Various transformations for inputs to produce the desired outputs.

  From earlier example:
  * Inputs: `x`
  * Variables: `W`, `b`
  * Outputs: `h`
  * Operations: `tf.nn.sigmoid(...)`, `tf.matmul(...)`

  Client can mainly receive data in three different ways:
  1. Feeding data at every step of the algorithm with Python code
  2. Preloading and storing data as TensorFlow tensors
  3. Building an input pipeline

# Feeding data with Python Code
In earlier example, `x` is an example of this method.

To feed data into the client from external data structures (e.g. `numpy.ndarray`) TensorFlow provides the `tf.placeholder`. Placeholders do not require actual data at the graph building stage. Rather, data is fed only for graph executions invoked with `session.run(..., feed_dict={placeholder: value})` by passing the external data to the `feed_dict` argument in the form of a Python dictionary where the key is the `tf.placeholder` variable and the corresponding value is the actual data (e.g. `numpy.ndarray`). Placeholder definition takes the form:

`tf.placeholder(dtype, shape=None, name=None)`

The arguments are:
* `dtype`: The data type for the data fed into the placeholder
* `shape`: Shape of the placeholder, given as a 1D vector
* `name`: Name of placeholder, important for debugging.



# Preloading and storing data as tensors
Similar to first method, but we do not have to data during graph execution as the data is preloaded. 

Two main differences:
1. When defining our input `x` we use `tf.constant`, assigning a specific value straight away and defining `x` as a tensor. 
2. We do not feed any additional input to `session.run()`. This is because our data has already been defined and thus we do not need to use `feed_dict`.

In [0]:
# Let us evaluate the sigmoid function using fixed values
preload_graph = tf.Graph()
preload_sess = tf.InteractiveSession(graph=graph)

# We instead define x as a tensor that contains specific values
x = tf.constant(value=[[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]], dtype=tf.float32, name='x')

# Same variables
W = tf.Variable(tf.random_uniform(shape=[10, 5], minval=-0.1, maxval=0.1, dtype=tf.float32), name='W')
b = tf.Variable(tf.zeros(shape=[5], dtype=tf.float32), name='b')

# Output
h = tf.nn.sigmoid(tf.matmul(x, W) + b)

# Executing operations and evaluating nodes in graph
tf.global_variables_initializer().run()

/tensorflow-1.15.0/python3.6/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
# Then we would not need to use feed_dict when running the session
preload_h_eval = preload_sess.run(h)
print(preload_h_eval)
preload_sess.close()

[[0.4783498  0.5351755  0.5512288  0.48623568 0.47490183]]


# Building an Input Pipeline
Designed for more heavy-duty clients that need to process a lot of data quickly. Creates a queue that holds data until it is needed. Possible to have multiple threads reading and processing in parallel. 

Typical pipeline will consist of:
* List of filenames
* Filename queue producing filenames for an input (record) reader
* Record reader for reading the inputs
* Decoder to decode the read records (e.g. JPEG image decoding)
* Preprocessing steps (optional)
* An example (decoded inputs) queue

In [0]:
# In this example we have three text files in CSV format, each with five lines and each line having 10 numbers separated by commas
# Need to read this data as batches (multiple rows of data vectors) by forming an input pipeline from files all the way to a tensor representing those input files
import tensorflow as tf
import numpy as np

# Create graph and session
graph = tf.Graph()
session = tf.InteractiveSession(graph=graph)

# Define filename queue (queue data structure containing filenames)
# Queue will produce filenames as requested by the reader, so that the reader can fetch the files with filenames to read data
filenames = ['test%d.txt'%i for i in range(1, 4)]
filename_queue = tf.train.string_input_producer(filenames, capacity=3, shuffle=True, name='string_input_producer')
# capacity is the amount of data held in the queue at a given time
# shuffle tells the queue if the data should be shuffled before spitting out

# TF has several readers
# As we have a few separate text files where a single line represents a single data point
reader = tf.TextLineReader()

# Use the read function to read data from the file, it outputs (key, value) pairs
key, value = reader.read(filename_queue, name='text_read_op')
